In [3]:
# Import libraries
import pandas as pd
import numpy as np

In [5]:
# Import 3 tables from IMDb datasets that we need
title_basics = pd.read_csv('data/title.basics.tsv.gz', sep='\t', header=0)
title_crew = pd.read_csv('data/title.crew.tsv.gz', sep='\t', header=0)
title_ratings = pd.read_csv('data/title.ratings.tsv.gz', sep='\t', header=0)

C:\Users\gideo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# Select only movies from the title_basics table, and convert the start years to numeric type
movies = title_basics[(title_basics.titleType == 'movie')].copy()
movies = movies[pd.to_numeric(movies['startYear'], errors='coerce').notnull()]
movies.startYear = pd.to_numeric(movies.startYear)

In [7]:
# Set the indicies for our 3 tables to the IMDb key for movies
movies.set_index('tconst', inplace=True)
title_crew.set_index('tconst', inplace=True)
title_ratings.set_index('tconst', inplace=True)

In [8]:
# Joint the 3 tables by tconst, the IMDb key for all movies
movies_crew = movies.join(title_crew, how='inner')
movies_crew_rating = movies_crew.join(title_ratings, how='inner')

In [9]:
# Example row from our new dataframe displaying information for 'The Shawshank Redemption'
movies_crew_rating[movies_crew_rating.primaryTitle == 'The Shawshank Redemption']

,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,directors,writers,averageRating,numVotes
tconst,,,,,,,,,,,,
tt0111161,movie,The Shawshank Redemption,The Shawshank Redemption,0,1994,\N,142,Drama,nm0001104,"nm0000175,nm0001104",9.3,2057323


**Note**: We can get the actual names (as opposed to their IDs on IMDb) of the writers and directors through the name_basics table. We can also get this information from the IMDbPY API.

Some movies have multiple directors (and almost every movie has multiple writers). So we would need to account for this in order to get the name in movies_crew_rating, if that is something we need. Currently, we don't need the actual names of directors and writers so we will not apply the needed logic.

In [10]:
# Take only the top 10,000 movies, where we rank movies by the number of votes they have received
top_10000_movies = movies_crew_rating.sort_values(by='numVotes', ascending=False)[:10000]
movies_index = top_10000_movies.index

In [11]:
from tqdm import tqdm
from time import sleep
from imdb import IMDb

ia = IMDb()

##### Code to get plot summaries 

We have already executed this code, so it is commented out. In the code that follows, we load the results into dataframe called 'plots'.

In [10]:
# Plot for-loop: This for-loop retrieved 10,000 plots from IMDb for us. All we needed to do is give it the IMDb ID of a movie
# and then it could retrieve the plot for us. We stored this plot in a dictionary where the keys are the movie IDs (tconst) and 
# the values are the movie plots

# This code is commented out because we have already run it and stored it in a CSV that we can easily import from.

# plots_dict = {}

# for movie_index in tqdm(movies_index):
#     sleep(1.5)
#     try:
#         plots_dict[movie_index] = ia.get_movie(movie_index[2:])['plot'][0]
#     except:
#         plots_dict[movie_index] = ''

## Convert out dictionary to a Dataframe and raname our column to 'plot'

# plots = pd.DataFrame.from_dict(movie_dict, orient='index')
# plots.rename(columns={0:'plot'}, inplace=True)

## Fill in the plots for any movies that we missed when calling the API the first time.
## Call the API as many times as neccessary to get the movies with missing plots

# missing_inds = [i for i in np.where(pd.isnull(plots))[0]]
# while len(missing_inds) != 0:
#     missing_movies = plots.iloc[missing_inds].index.values.tolist()
#     for missing_movie in missing_movies:
#         plots.loc[missing_movie] = ia.get_movie(missing_movie[2:])['plot'][0]
#     missing_inds = [i for i in np.where(pd.isnull(plots))[0]]

## Save the plots to a CSV
# plots.to_csv(path_or_buf='movies.csv')

In [11]:
# Load our plots for the CSV
plots = pd.read_csv('movies.csv')
plots.rename(columns={'Unnamed: 0':'tconst'}, inplace=True)
plots.set_index('tconst', inplace=True)

In [12]:
# Merge plot into our movies table
movies = top_10000_movies.join(plots, how='inner')

In [13]:
movies.drop(labels=['titleType', 'originalTitle', 'isAdult', 'endYear'], axis='columns', inplace=True) 
# We can get rid of the following columns:
#1) 'titleType': know we only have movies, so don't need this columns
#2) 'originalTitle': We don't really care what the original title was, even if it is different from its primary title
#3) 'isAdult': None of these movies are adult, so we can drop
#4) 'endYear': All of the endYear values are '\N', so we can get rid of this column 

##### Code to get Plot Outlines

Try and implement the below code by writing directly to a CSV instead of using pandas and possiby even dicts (i.e. can we write to a CSV directly?)

In [14]:
plot_outlines= {}

for movie_index in tqdm(movies_index):
    sleep(1.5)
    movie = ia.get_movie(movie_index[2:])
    try:
        plot_outlines[movie_index] = movie['plot outline']
    except:
        plot_outlines[movie_index] = ''

## Convert out dictionary to a Dataframe and raname our column to 'plot'

plot_outlines = pd.DataFrame.from_dict(plot_outlines, orient='index')
plot_outlines.rename(columns={0:'plot'}, inplace=True)

## Save the plots to a CSV
plot_outlines.to_csv(path_or_buf='plot_outlines.csv')

100%|██████████| 10000/10000 [8:35:19<00:00,  2.93s/it]  


In [39]:
# Load our plot outlies for the CSV
plot_outlines = pd.read_csv('plot_outlines.csv')
plot_outlines.rename(columns={'Unnamed: 0':'tconst'}, inplace=True)
plot_outlines.set_index('tconst', inplace=True)
plot_outlines.rename(columns={'plot':'plot outlines'}, inplace=True)

In [55]:
# Join to our table
movies = movies.join(plot_outlines, how='inner')
# Save our dataframe 
movies.to_csv(path_or_buf='movies0.csv')

##### Get Keywords

In [126]:
# keywords_dict = {}
# for movie_index in tqdm(movies_index):
#     sleep(1)
#     try:
#         keywords_dict[movie_index] = ia.get_movie_keywords(movie_index[2:])['data']['keywords']
#     except:
#         keywords_dict[movie_index] = ''   
        
# keywords = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in keywords_dict.items() ])).transpose()
# keywords = keywords.apply(lambda x: ','.join(x.dropna()), axis=1)
# keywords = pd.DataFrame(keywords)
# keywords.rename(columns={0:'keywords'}, inplace=True)

# ## Save the plots to a CSV
# keywords.to_csv(path_or_buf='keywords.csv')

In [129]:
# Load our plot outlies for the CSV
keywords = pd.read_csv('keywords.csv')
keywords.rename(columns={'Unnamed: 0':'tconst'}, inplace=True)
keywords.set_index('tconst', inplace=True)

In [139]:
# Join to our table
movies = movies.join(keywords, how='inner')
# Save our dataframe 
movies.to_csv(path_or_buf='movies0.csv')

##### Get Synopses

In [13]:
# synopses_dict = {}
# for movie_index in tqdm(movies_index):
#     sleep(2)
#     try:
#         synopses_dict[movie_index] = ia.get_movie(movie_index[2:])['synopsis']
#     except:
#         synopses_dict[movie_index] = ''   

# ## Convert out dictionary to a Dataframe and raname our column to 'synopsis'
# synopses = pd.DataFrame.from_dict(synopses_dict).transpose()
# synopses.rename(columns={0:'synopsis'}, inplace=True)

# ## Save the plots to a CSV
# synopses.to_csv(path_or_buf='synopses.csv')

100%|██████████| 10000/10000 [9:57:49<00:00,  3.40s/it]  


In [24]:
# Load synopses 
synopses = pd.read_csv('synopses.csv')
synopses.rename(columns={'Unnamed: 0':'tconst'}, inplace=True)
synopses.set_index('tconst', inplace=True)

In [63]:
# Join to our table
movies = movies.join(synopses, how='inner')
# Save our dataframe 
movies.to_csv(path_or_buf='movies0.csv')